# Preprocessing

In [1]:
import json
import uuid

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from difflib import SequenceMatcher
from IPython.display import HTML

from utils.code_utils import tokenize_python_code

pd.set_option('display.max_columns', None)

codenet_root = 'datasets/codenetpy/'

In [2]:
with open(codenet_root + 'codenetpy_train.json', 'r') as f:
    data = json.load(f)["data"]
    
with open(codenet_root + 'codenetpy_test.json', 'r') as f:
    test_data = json.load(f)["data"]
    

train_df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)

codenetpy_df = pd.DataFrame(data + test_data)

display(codenetpy_df)
display(codenetpy_df.info())

,original_src,changed_src,problem_id,original_id,changed_id,language,filename_ext,original_status,returncode,error_class,error_class_extra,error,output
0,l=[]\nwhile True:\n try:\n l.append(...,l=[]\nwhile True:\n try:\n l.append(...,p00001,s196059089,s508355022,Python,py,Runtime Error,1,SyntaxError,SyntaxError: Missing parentheses in call to 'p...,"File ""/home/alex/Documents/research/bug-dete...",
1,"data = []\nfor i in range(0,10):\n data.app...","data = []\nfor i in range(0,10):\n data.app...",p00001,s840313913,s627573591,Python,py,WA: Presentation Error,0,0,,,\n3776\n2848\n2840\n
2,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,p00001,s973623080,s521689663,Python,py,Runtime Error,1,TypeError,TypeError: '>' not supported between instances...,"Traceback (most recent call last):\n File ""/h...",
3,for i in range(10):\n s.append(int(input())...,s=[]\nfor i in range(10):\n s.append(int(in...,p00001,s393051088,s459982723,Python,py,Runtime Error,1,NameError,NameError: name 's' is not defined,"Traceback (most recent call last):\n File ""/h...",
4,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,p00001,s803831828,s403126268,Python,py,Runtime Error,1,NameError,NameError: name 'argv' is not defined,"Traceback (most recent call last):\n File ""/h...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54587,"N = int(input())\nL = list(map(int, input().sp...","N = int(input())\nL = list(map(int, input().sp...",p04047,s420396881,s036877571,Python,py,Runtime Error,1,IndexError,IndexError: list index out of range,"Traceback (most recent call last):\n File ""/h...",
54588,\nnum = int(input())\nli = int(input().split()...,n = int(input())\nli = [int(x) for x in input(...,p04047,s838662080,s028090472,Python,py,Runtime Error,1,TypeError,"TypeError: int() argument must be a string, a ...","Traceback (most recent call last):\n File ""/h...",
54589,from collections import Counter\n \nn = int(in...,"n = int(input())\n \narr = list(map(int, input...",p04047,s112597968,s888016571,Python,py,Runtime Error,1,TypeError,TypeError: unsupported operand type(s) for +=:...,"Traceback (most recent call last):\n File ""/h...",
54590,"N,X = input().split()\nN,X = int(N), int(X)\na...","N,X = input().split()\nN,X = int(N), int(X)\na...",p04048,s311100241,s323769427,Python,py,Runtime Error,1,TabError,TabError: inconsistent use of tabs and spaces ...,"File ""/home/alex/Documents/research/bug-dete...",


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54592 entries, 0 to 54591
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_src       54592 non-null  object
 1   changed_src        54592 non-null  object
 2   problem_id         54592 non-null  object
 3   original_id        54592 non-null  object
 4   changed_id         54592 non-null  object
 5   language           54592 non-null  object
 6   filename_ext       54592 non-null  object
 7   original_status    54592 non-null  object
 8   returncode         54592 non-null  int64 
 9   error_class        54592 non-null  object
 10  error_class_extra  54592 non-null  object
 11  error              54592 non-null  object
 12  output             54592 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.4+ MB


None

In [3]:
codenetpy_df['error_class'].value_counts()

0                            28406
SyntaxError                   8686
NameError                     5262
TypeError                     4202
ValueError                    2444
IndentationError              1081
IndexError                     999
AttributeError                 913
EOFError                       852
TLEError                       525
ModuleNotFoundError            357
TabError                       265
ImportError                    132
ZeroDivisionError               79
KeyError                        66
SyntaxWarning                   62
FileNotFoundError               62
UnboundLocalError               61
1                               58
DeprecationWarning              35
RecursionError                  13
OverflowError                   12
RuntimeError                     6
-11                              5
2                                3
OSError                          3
SparseEfficiencyWarning          1
255                              1
VisibleDeprecationWa

In [4]:
codenetpy_df['returncode'].value_counts()

 0      28455
 1      25603
-9        525
-11         5
 2          3
 255        1
Name: returncode, dtype: int64

In [5]:
# There are cases where the return code is 0 but the error_class is a Warning

codenetpy_df[(codenetpy_df['returncode'] == 0) & (codenetpy_df['error_class'] != '0')]['error_class'].value_counts()

DeprecationWarning         30
SyntaxWarning              18
SparseEfficiencyWarning     1
Name: error_class, dtype: int64

In [6]:
# Most of the '0' error_class buggy submissions are TLE errors, which means they are inefficient implementations

codenetpy_df[codenetpy_df['error_class'] == '0']['original_status'].value_counts()

Time Limit Exceeded       17252
Runtime Error              9590
WA: Presentation Error     1485
Memory Limit Exceeded        73
Output Limit Exceeded         5
Judge Not Available           1
Name: original_status, dtype: int64

In [7]:
# Idealy the model will take as input the original_src code and will identify both the error_description and the location of the bug
# We can achieve this by comparing the original_src with the changed_src, which will give us the location of the bug (e.g. the modification)
# Next we will want to create a new model that will take as input the original_src, and the found bug and will learn to make the modifications
# on the original code such that it gets accepted

buggy_df = codenetpy_df[codenetpy_df['returncode'] != 0]
buggy_df.rename(columns={'original_src': 'code_string', 'error_class_extra': 'error_msg'}, inplace=True)
buggy_df = buggy_df[['code_string', 'error_msg']]
buggy_df['uuid'] = [uuid.uuid4().hex for _ in range(len(buggy_df.index))]
buggy_df = buggy_df.set_index('uuid')
buggy_df


/Users/angelchavez/opt/miniconda3/envs/BIFI/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,code_string,error_msg
uuid,,
2949d1ea56bb46b2b757e323cbe00ac1,l=[]\nwhile True:\n try:\n l.append(...,SyntaxError: Missing parentheses in call to 'p...
884ad79786d648c8a454a9df85590418,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,TypeError: '>' not supported between instances...
f708055d2e004141b544de41914df2b1,for i in range(10):\n s.append(int(input())...,NameError: name 's' is not defined
ec74fe8124ad48f09898fa4539b597e3,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,NameError: name 'argv' is not defined
702bc130f430474387918079e571d9e9,height = []\nwhile 1:\n h = raw_input()\n ...,SyntaxError: Missing parentheses in call to 'p...
...,...,...
0a61d10b72004c83ad27521b69f6aecc,"N = int(input())\nL = list(map(int, input().sp...",IndexError: list index out of range
54938d6732994f84a56fbcab2c4d0420,\nnum = int(input())\nli = int(input().split()...,"TypeError: int() argument must be a string, a ..."
11926787f12d48cb9699befaa1645c0c,from collections import Counter\n \nn = int(in...,TypeError: unsupported operand type(s) for +=:...


In [8]:
buggy_df.to_json(r'preprocess/raw.json', orient='index', indent=4)

# Tokenizing

In [9]:
raw_df = pd.read_json('preprocess/raw.json', orient='index')
#raw_df['code_toks_joined'], raw_df['anonymize_dict'] = zip(*raw_df['code_string'].map(tokenize_python_code))
raw_df[['code_toks_joined', 'anonymize_dict']] = raw_df.apply(lambda row: tokenize_python_code(row['code_string']), axis='columns', result_type='expand')
raw_df['code_toks_joined'] = raw_df['code_toks_joined'].str.join(" ")
raw_df.head()


('ERRORTOKEN', (2, 40))
('ERRORTOKEN', (7, 20))
('ERRORTOKEN', (6, 8))
('ERRORTOKEN', (5, 13))
('ERRORTOKEN', (3, 0))
('ERRORTOKEN', (5, 8))
('ERRORTOKEN', (7, 0))
('ERRORTOKEN', (4, 0))
('ERRORTOKEN', (16, 6))
('ERRORTOKEN', (7, 39))
('ERRORTOKEN', (12, 8))
('ERRORTOKEN', (7, 9))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (4, 9))
('ERRORTOKEN', (12, 40))
('ERRORTOKEN', (3, 4))
('ERRORTOKEN', (6, 27))
('ERRORTOKEN', (3, 19))
('ERRORTOKEN', (1, 53))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (16, 27))
('ERRORTOKEN', (4, 31))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (1, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (1, 2))
('ERRORTOKEN', (4, 6))
('ERRORTOKEN', (12, 31))
('ERRORTOKEN', (13, 78))
('ERRORTOKEN', (1, 19))
('ERRORTOKEN', (5, 0))
('ERRORTOKEN', (14, 27))
('ERRORTOKEN', (17, 0))
('ERRORTOKEN', (2, 0))
('ERRORTOKEN', (5, 11))
('ERRORTOKEN', (2, 27))
('ERRORTOKEN', (22, 21))
('ERRORTOKEN', (20, 4))
('EOF in multi-line string', (

,code_string,error_msg,code_toks_joined,anonymize_dict
2949d1ea56bb46b2b757e323cbe00ac1,l=[]\nwhile True:\n try:\n l.append(...,SyntaxError: Missing parentheses in call to 'p...,l = [ ] <NEWLINE> while True : <NEWLINE> <INDE...,{}
884ad79786d648c8a454a9df85590418,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,TypeError: '>' not supported between instances...,first = 0 ; <NEWLINE> second = 0 ; <NEWLINE> t...,{}
f708055d2e004141b544de41914df2b1,for i in range(10):\n s.append(int(input())...,NameError: name 's' is not defined,for i in range ( 10 ) : <NEWLINE> <INDENT> s ....,{}
ec74fe8124ad48f09898fa4539b597e3,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,NameError: name 'argv' is not defined,<COMMENT> <NL> <NL> import sys <NEWLINE> <NL> ...,"{'<COMMENT>': ['# -*- coding: utf-8 -*-'], '<S..."
702bc130f430474387918079e571d9e9,height = []\nwhile 1:\n h = raw_input()\n ...,SyntaxError: Missing parentheses in call to 'p...,height = [ ] <NEWLINE> while 1 : <NEWLINE> <IN...,{'<STRING>': ['''']}


In [10]:
raw_df.to_json(r'preprocess/tokenized.json', orient='index', indent=4)


## Tratamiento adicional de JSON

In [11]:
with open('preprocess/tokenized.json', 'r') as f:
    data = json.load(f)
data


{'2949d1ea56bb46b2b757e323cbe00ac1': {'code_string': 'l=[]\nwhile True:\n    try:\n        l.append(input())\n    except SyntaxError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]',
  'error_msg': "SyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?",
  'code_toks_joined': 'l = [ ] <NEWLINE> while True : <NEWLINE> <INDENT> try : <NEWLINE> <INDENT> l . append ( input ( ) ) <NEWLINE> <DEDENT> except SyntaxError : <NEWLINE> <INDENT> break <NEWLINE> <DEDENT> <DEDENT> l . sort ( reverse = True ) <NEWLINE> for i in range ( 3 ) : <NEWLINE> <INDENT> print l [ i ] <NEWLINE> <DEDENT>',
  'anonymize_dict': {}},
 '884ad79786d648c8a454a9df85590418': {'code_string': 'first = 0;\nsecond = 0;\nthird = 0;\nfor var in range(0,10):\n\tN = input()\n\tif (N>first):\n\t\tthird = second\n\t\tsecond = first\n\t\tfirst = N\n\telif (N>second):\n\t\tthird = second\n\t\tsecond = N\n\telif (N>third):\n\t\tthird = N\nprint (first)\nprint (second)\nprint (third

In [12]:
for key, obj in data.items():
    for attribute, value in obj.items():
        if attribute == 'error_msg':
            new_obj = { 'msg': value }
            obj[attribute] = new_obj
            
# Serializing json
json_object = json.dumps(data, indent=4)
 
# Writing to sample.json
with open("preprocess/final.json", "w") as outfile:
    outfile.write(json_object)